In [32]:
#https://www.kaggle.com/anikannal/solar-power-generation-data/tasks

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
from sklearn.metrics import median_absolute_error
from sklearn.linear_model import Ridge
from category_encoders import OneHotEncoder
from sklearn.pipeline import make_pipeline

In [5]:
train = pd.read_parquet("train.parquet")
val = pd.read_parquet("val.parquet")

# Supervised embeddings

In [6]:
train2 = train.copy()
train2['WEEKDAY'] = train2['DATE_TIME'].dt.weekday
train2['HOUR'] = train2['DATE_TIME'].dt.hour
train2['MINUTE'] = train2['DATE_TIME'].dt.minute


val2 = val.copy()
val2['WEEKDAY'] = val2['DATE_TIME'].dt.weekday
val2['HOUR'] = val2['DATE_TIME'].dt.hour
val2['MINUTE'] = val2['DATE_TIME'].dt.minute



In [7]:
from category_encoders import OrdinalEncoder
encoder = OrdinalEncoder(cols=['SOURCE_KEY'])
train2['SOURCE_KEY_ENCODED'] = encoder.fit_transform(train2[['SOURCE_KEY']]) + 1
val2['SOURCE_KEY_ENCODED'] = encoder.transform(val2[['SOURCE_KEY']]) + 1

In [8]:
cats = ['SOURCE_KEY_ENCODED', 'WEEKDAY','HOUR', 'MINUTE']

In [9]:
import torch.nn as nn
import torch
import torch.optim as opt
import torch.nn.functional as F

# Todos os embeddings
- um para cada feature ou 1 para a combinação? teste!

In [10]:
class EmbeddingNet2(nn.Module):
    def __init__(self):
        super(EmbeddingNet2, self).__init__()
        self.e_source_key = nn.Embedding(24, 8, padding_idx=0)
        self.e_weekday = nn.Embedding(7, 8, padding_idx=0)
        self.e_hour = nn.Embedding(24, 8, padding_idx=0)
        self.e_minute = nn.Embedding(60, 8, padding_idx=0)
        
        self.hid1 = nn.Linear(32, 100)
        self.hid2 = nn.Linear(100, 100)
        self.hid3 = nn.Linear(100, 100)
        self.out = nn.Linear(100, 1)
    
    def forward(self, x, return_embeddings=False, return_hid3=False):
        e_source_key = self.e_source_key(x[:,0])
        e_weekday = self.e_weekday(x[:,1])
        e_hour = self.e_hour(x[:,2])
        e_minute = self.e_minute(x[:,3])
        
        e = torch.cat([e_source_key, e_weekday, e_hour, e_minute], dim=-1)
        #print(x.shape)
        x = F.relu(self.hid1(e))
        x = F.relu(self.hid2(x))
        
       
        hid3 = self.hid3(x)
        x = F.relu(hid3)
        x = self.out(x)
        
        if return_embeddings:
            #return e_source_key, e_weekday, e_hour, e_minute
            return e
        elif return_hid3:
            return hid3
        return x
        

In [13]:
torch.manual_seed(0)
net = EmbeddingNet2().cuda()
criterion = nn.L1Loss()
optimizer = opt.Adam(net.parameters(), lr=1e-4)

x = torch.from_numpy(train2[cats].values).cuda()
y = torch.from_numpy(train2['Y4WIN'].values.reshape(-1,1))
y_cuda = y.cuda()
for e in range(10000):
    
    optimizer.zero_grad() # tinha esquecido de colocar, modelo não convergia
    p = net(x).cuda()
    loss = criterion(p, y_cuda)
    #print(loss.item())
    
    loss.backward()
    optimizer.step()
    
    p_numpy = p.cpu().detach().numpy()
    if e % 1000 == 0:
        print(median_absolute_error(y, p_numpy))
        
#diverge depois converge

13.675929112028102
3.6717558654481763
1.8563446998596191
1.382433533668518
1.1050314903259277
0.913633135795215
0.8508844001921716
0.7512656005555982
0.7044656080390581
0.6773707492980066


# Embedding to Tree (Leaky)

In [17]:
Xtr.shape

(33112, 32)

In [18]:
Xval.shape

(35578, 32)

In [14]:
from sklearn.ensemble import RandomForestRegressor
mdl = RandomForestRegressor(n_jobs=-1, random_state=0, n_estimators=100)

x = torch.from_numpy(train2[cats].values).cuda()
Xtr = net(x,return_embeddings=True).detach().cpu().numpy()
ytr = train2['Y4WIN']
mdl.fit(Xtr,ytr)

x = torch.from_numpy(val2[cats].fillna(0).values).long().cuda()
Xval =  net(x,return_embeddings=True).detach().cpu().numpy()

p = mdl.predict(Xval)
median_absolute_error(val2['Y4'], p)

2.5916437232333855

# Hidden to Tree (Leaky)

In [19]:
from sklearn.ensemble import RandomForestRegressor
mdl = RandomForestRegressor(n_jobs=-1, random_state=0, n_estimators=100)

x = torch.from_numpy(train2[cats].values).cuda()
Xtr = net(x,return_hid3=True).detach().cpu().numpy()
ytr = train2['Y4WIN']
mdl.fit(Xtr,ytr)

x = torch.from_numpy(val2[cats].fillna(0).values).long().cuda()
Xval =  net(x,return_hid3=True).detach().cpu().numpy()

p = mdl.predict(Xval)
median_absolute_error(val2['Y4'], p)

2.719459821801583